# Loadout

In [1]:
# import sys
# sys.path.append('..')

# %load_ext autoreload
# %autoreload 2




In [2]:
import panel as pn
from dotenv import load_dotenv
import param

load_dotenv()

pn.extension()

In [3]:
# Try to use --design-... variables first
pn.config.global_css = [
    """
@import url('https://fonts.googleapis.com/css2?family=Hanken+Grotesk:ital,wght@0,100..900;1,100..900&family=Ubuntu:ital,wght@0,300;0,400;0,500;0,700;1,300;1,400;1,500;1,700&display=swap');
body {
    --primary-background-color: #0C1314;
    --secondary-background-color: #111926;
    --light-outline-color: #F4F6F8;
    --primary-accent-color: #D33A4B;
    --secondary-accent-color: #EDB737;
    --tertiary-accent-color: #12B86C;
    --white: #F4F6F8;

    --base-font: 'Hanken Grotesk', sans-serif;
    --bokeh-base-font: 'Hanken Grotesk', sans-serif;
    --bokeh-font-size: 16px;
    --design-background-text-color: var(--white);

    
    background-color: var(--primary-background-color);
    /* Centering Body */
    display: flex;
    justify-content: center;
    align-items: center;
}
"""
]

In [4]:
# srv.stop()

In [5]:
# from pyllments.payloads.message import MessagePayload
# from langchain_core.messages import HumanMessage

# msg =MessagePayload(
#     message=HumanMessage(content='Hello'), role='human'
# ).create_message_view()
# srv = pn.serve(msg)
# msg

In [6]:
# # Create a toggle button to expand/collapse the content
# toggle = pn.widgets.Toggle(icon='chevron-down', sizing_mode='stretch_height', name='Expand', button_type='primary')

# # Create the initial markdown content with a short preview
# initial_content = """
# ## Expanded Content
# This is a short preview of the content...
# """

# # Create the full markdown content to be displayed when expanded
# full_content = """
# ## Expanded Content

# This is the full content that will be shown when the toggle is activated. 
# It contains additional information and details...
# """

# # Add a CSS stylesheet for improved transition effects
# transition_css = """
# .expandable-content {
#     transition: max-height 0.5s ease-out, opacity 0.5s ease-out;
#     max-height: 50px;
#     opacity: 0.7;
#     overflow: hidden;
# }
# .expandable-content.expanded {
#     max-height: 1000px;
#     opacity: 1;
# }
# """

# # Create a markdown pane for the content
# expandable_content = pn.pane.Markdown(initial_content, sizing_mode='stretch_width', css_classes=['expandable-content'])

# # Create a function to toggle the visibility of the content and update the text
# def toggle_visibility(event):
#     if event.new:  # If the toggle is activated
#         expandable_content.object = full_content
#         expandable_content.css_classes = ['expandable-content', 'expanded']
#     else:  # If the toggle is deactivated
#         expandable_content.object = initial_content
#         expandable_content.css_classes = ['expandable-content']

# # Link the toggle button to the visibility function
# toggle.param.watch(toggle_visibility, 'value')

# # Create the row layout with the toggle button and expandable content
# row = pn.Row(
#     toggle,
#     expandable_content,
#     sizing_mode='stretch_width'
# )

# # Apply the custom CSS
# pn.extension(raw_css=[transition_css])

# # Display the row
# srv = pn.serve(row.servable())
# row

In [7]:
from pyllments.logging import setup_logging
setup_logging(log_file='main.log', stdout_log_level='DEBUG', file_log_level='INFO')
from loguru import logger

from pyllments.elements.chat_interface import ChatInterfaceElement
from pyllments.elements.llm_chat import LLMChatElement

from pyllments.elements.context_builder import ContextBuilderElement

# Main

In [29]:
class CC(param.Parameterized):
    """A custom parameterized class for demonstration purposes."""
    
    # Define parameters for the class
    example_param = param.String(default='default_value', doc="An example parameter.")

    def __init__(self, **params):
        super().__init__(**params)
          # Set the name of the instance


In [33]:
c = CC(name='waaat')
c.name

'waaat'

In [6]:
chat_interface = ChatInterfaceElement()

llm_chat = LLMChatElement(output_mode='stream')

chat_interface.ports.output['message_output'] > llm_chat.ports.input['messages_input']
llm_chat.ports.output['message_output'] > chat_interface.ports.input['message_input']


In [7]:
chatfeed_view = chat_interface.create_chatfeed_view(height=300, auto_scroll_limit=1, width=500)
send_button_view = chat_interface.create_send_button_view(width=70, height=70)
chat_input_view = chat_interface.create_chat_input_view(width=400)

pn.Column(
    chatfeed_view,
    pn.Row(chat_input_view, send_button_view)
).servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'6cebad53-76b8-49c9-a3a7-cadbdb1a2324': {'version…

2024-07-25 01:52:48.594 | INFO     | Staging: ChatInterfaceElement | message_output | Staged Item: new_message: MessagePayload
2024-07-25 01:52:48.594 | INFO     | Emitting from ChatInterfaceElement | Port: message_output | Payload: MessagePayload
2024-07-25 01:52:48.595 | INFO     | Receiving in LLMChatElement | Port: messages_input | Payload: MessagePayload
2024-07-25 01:52:48.595 | INFO     | Staging: LLMChatElement | message_output | Staged Item: outgoing_message: generator
2024-07-25 01:52:48.595 | INFO     | Emitting from LLMChatElement | Port: message_output | Payload: MessagePayload
2024-07-25 01:52:48.596 | INFO     | Receiving in ChatInterfaceElement | Port: message_input | Payload: MessagePayload


1

In [5]:
chat_interface = ChatInterfaceElement() 

llm_chat = LLMChatElement(output_mode='stream')

context_builder = ContextBuilderElement()

chat_interface.ports.output['message_output'] > \
    context_builder.ports.input['message_input']

context_builder.ports.output['messages_output'] > \
    llm_chat.ports.input['messages_input']

llm_chat.ports.output['message_output'] > (
    chat_interface.ports.input['message_input'],
    context_builder.ports.input['message_input']
    )

chatfeed_view = chat_interface.create_chatfeed_view(height=300, auto_scroll_limit=1, width=500)
send_button_view = chat_interface.create_send_button_view(width=70, height=70)
chat_input_view = chat_interface.create_chat_input_view(width=400)
context_view = context_builder.create_context_view()

interface = pn.Column(
    pn.Row(chatfeed_view, context_view),
    pn.Row(chat_input_view, send_button_view),
    
).servable()
interface

BokehModel(combine_events=True, render_bundle={'docs_json': {'ff2e312d-8ee8-40f2-8adc-194193dcb506': {'version…

2024-07-28 18:36:04.221 | WARNING  | CSS file not found: /workspaces/pyllments/pyllments/elements/chat_interface/css/payload_human_markdown.css
2024-07-28 18:36:04.223 | WARNING  | CSS file not found: /workspaces/pyllments/pyllments/elements/chat_interface/css/payload_human_row.css
2024-07-28 18:36:04.224 | WARNING  | CSS file not found: /workspaces/pyllments/pyllments/elements/chat_interface/css/payload_ai_markdown.css
2024-07-28 18:36:04.226 | WARNING  | CSS file not found: /workspaces/pyllments/pyllments/elements/chat_interface/css/payload_ai_row.css
2024-07-28 18:36:04.240 | INFO     | Staging: ChatInterfaceElement | message_output | Staged Item: new_message: MessagePayload
2024-07-28 18:36:04.241 | INFO     | Emitting from ChatInterfaceElement | Port: message_output | Payload: MessagePayload
2024-07-28 18:36:04.241 | INFO     | Receiving in ContextBuilderElement | Port: message_input | Payload: MessagePayload
2024-07-28 18:36:04.471 | INFO     | Staging: ContextBuilderElement | me

In [8]:
srv = pn.serve(interface)

Launching server at http://localhost:37055


2024-07-28 05:33:54.229 | WARNING  | CSS file not found: /workspaces/pyllments/pyllments/elements/chat_interface/css/payload_human_markdown.css
2024-07-28 05:33:54.232 | WARNING  | CSS file not found: /workspaces/pyllments/pyllments/elements/chat_interface/css/payload_human_row.css
2024-07-28 05:33:54.233 | WARNING  | CSS file not found: /workspaces/pyllments/pyllments/elements/chat_interface/css/payload_ai_markdown.css
2024-07-28 05:33:54.235 | WARNING  | CSS file not found: /workspaces/pyllments/pyllments/elements/chat_interface/css/payload_ai_row.css
2024-07-28 05:33:54.243 | INFO     | Staging: ChatInterfaceElement | message_output | Staged Item: new_message: MessagePayload
2024-07-28 05:33:54.244 | INFO     | Emitting from ChatInterfaceElement | Port: message_output | Payload: MessagePayload
2024-07-28 05:33:54.244 | INFO     | Receiving in ContextBuilderElement | Port: message_input | Payload: MessagePayload
2024-07-28 05:33:54.245 | INFO     | Staging: ContextBuilderElement | me

In [10]:
chat_interface.model.message_list[0].css_cache['message_view']

{'human_markdown': ":host {\n  --base-font: 'Hanken Grotesk', sans-serif;\n  color: #F4F6F8;\n  font-weight: 300;\n}\n\np {\n  font-family: var(--base-font);\n}",
 'human_row': ':host {\n    max-height: none; /* Fixes streaming issue in Panel */\n    background-color: #0C1314;\n    border-right: 2px solid #12B86C;\n    width: 100%;\n}',
 'ai_markdown': ":host {\n    --base-font: 'Hanken Grotesk', sans-serif;\n\n    color: #F4F6F8;\n    width: 80%;\n    margin: auto;\n}\n\np {\n    font-family: var(--base-font);\n}\n",
 'ai_row': ':host {\n  max-height: none;\n  align-items: center;\n  justify-content: center;\n  width: 100%;\n}'}

In [7]:
from pyllments.payloads.message import MessagePayload
MessagePayload()._get_module_path()

PosixPath('/workspaces/pyllments/pyllments/payloads/message')

In [7]:
# Define three stylesheets as strings to change the background color of the specified div
green_stylesheet = """.noUi-connect { 
    background-color: green !important;
}"""

red_stylesheet = """.noUi-connect {
    background-color: red !important;
}
    .noUi-handle {
    background-color: green !important;
}
"""



purple_stylesheet = """.noUi-connect {
    background-color: purple !important;
}"""

# These stylesheets can be applied to the relevant Panel components to change the background color of the div


In [11]:
pn.widgets.RangeSlider(start=0, end=10, step=1, value=(0, 10),
                                      stylesheets=['', '']).servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'4a0fdd29-5d66-445a-95c0-00b4d4bb1d5d': {'version…

In [27]:
srv = pn.serve(pn.widgets.RangeSlider(start=0, end=10, step=1, value=(0, 10),
                                      stylesheets=[green_stylesheet, red_stylesheet, purple_stylesheet ]))

Launching server at http://localhost:34563


In [26]:
srv.stop()

AssertionError: Already stopped

In [7]:
chat_interface = ChatInterfaceElement() 
context_builder = ContextBuilderElement() 

chat_interface.ports.output['message_output'] > context_builder.ports.input['message_input']

context_builder.model.context

chatfeed_view = chat_interface.create_chatfeed_view(height=300, auto_scroll_limit=1, width=500)
send_button_view = chat_interface.create_send_button_view(width=70, height=70)
chat_input_view = chat_interface.create_chat_input_view(width=400)
context_view = context_builder.create_context_view()

pn.Column(
    pn.Row(chatfeed_view, context_view),
    pn.Row(chat_input_view, send_button_view),
    
).servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'41ff5161-490c-4273-baaf-146bf39774db': {'version…

2024-07-26 23:55:51.933 | INFO     | Staging: ChatInterfaceElement | message_output | Staged Item: new_message: MessagePayload
2024-07-26 23:55:51.934 | INFO     | Emitting from ChatInterfaceElement | Port: message_output | Payload: MessagePayload
2024-07-26 23:55:51.934 | INFO     | Receiving in ContextBuilderElement | Port: message_input | Payload: MessagePayload
2024-07-26 23:55:52.148 | INFO     | Staging: ContextBuilderElement | messages_output | Staged Item: context: deque
2024-07-26 23:55:52.149 | INFO     | Emitting from ContextBuilderElement | Port: messages_output | Payload: list


In [5]:
from pyllments.payloads.message import MessagePayload
from langchain_core.messages import HumanMessage, AIMessage

mp = MessagePayload(message=HumanMessage(content='Hello'))
mp.create_message_view()


BokehModel(combine_events=True, render_bundle={'docs_json': {'4def90a0-f6f2-4969-a74b-fb272ec5b402': {'version…

In [8]:
from pyllments.payloads.message import MessagePayload
from langchain_core.messages import HumanMessage, AIMessage

ci = ChatInterfaceElement()

In [9]:
ci.model.message_list.extend([
    MessagePayload(message=HumanMessage(content='Hello'), role='human'),
    MessagePayload(message=AIMessage(content='Hello, I am AI'), role='ai'),
])

In [10]:
ci.create_chatfeed_view()

2024-07-29 05:51:29.265 | WARNING  | CSS file not found: /workspaces/pyllments/pyllments/elements/chat_interface/css/payload_human_markdown.css
2024-07-29 05:51:29.267 | WARNING  | CSS file not found: /workspaces/pyllments/pyllments/elements/chat_interface/css/payload_human_row.css
2024-07-29 05:51:29.268 | WARNING  | CSS file not found: /workspaces/pyllments/pyllments/elements/chat_interface/css/payload_ai_markdown.css
2024-07-29 05:51:29.270 | WARNING  | CSS file not found: /workspaces/pyllments/pyllments/elements/chat_interface/css/payload_ai_row.css


BokehModel(combine_events=True, render_bundle={'docs_json': {'58b124be-6a00-4d8a-9123-8947b4bf21c7': {'version…